<a href="https://colab.research.google.com/github/blacurly/MLOps/blob/main/Neptune_Athletes_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install neptune

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import neptune
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

random_state=20250701

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Quarter 4/MLOps/athletes_v2.csv")

In [ ]:
run = neptune.init_run(
    project="blacurly/MLOps",
    api_token="----",
    tags=["v1", "baseline","random-forest"],
    name="baseline-v2-tracking"
)

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs-legacy.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/blacurly/MLOps/e/MLOP-10


In [ ]:
features = ['age','height','weight', 'deadlift', 'backsq', 'candj', 'snatch']
target = 'total_lift'

# Split
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (24023, 7), Test shape: (6006, 7)


In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=random_state)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

##### Log Metrics to Neptune

In [ ]:
def evaluate_and_log_model(name, y_true, y_pred, run):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} Performance:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"R²  : {r2:.3f}")
    print("-" * 30)

    run[f"{name}/RMSE"] = rmse
    run[f"{name}/MAE"] = mae
    run[f"{name}/R2"] = r2

In [ ]:
evaluate_and_log_model("Random Forest", y_test, rf_pred, run)
run.stop()

Random Forest Performance:
RMSE: 10.77
MAE : 4.40
R²  : 0.998
------------------------------
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 3 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/blacurly/MLOps/e/MLOP-10/metadata
